In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader
ds = ParaphraserReader().read(data_path='dataset', do_lower_case=False)
texts_a = []
texts_b = []
labels = []
for (ta, tb), label in ds['test']:
    texts_a.append(ta)
    texts_b.append(tb)
    labels.append(label)

In [3]:
BATCH_SIZE = 64

In [4]:
from deeppavlov import build_model, configs

para_model = build_model(configs.classifiers.paraphraser_rubert, download=True)

2019-06-17 18:09:15.905 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/deeppavlov_data/classifiers/paraphraser_rubert_v0.tar.gz download because of matching hashes
I0617 18:09:15.905855 140398082541376 download.py:116] Skipped http://files.deeppavlov.ai/deeppavlov_data/classifiers/paraphraser_rubert_v0.tar.gz download because of matching hashes
2019-06-17 18:09:16.824 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
I0617 18:09:16.824452 140398082541376 download.py:116] Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2019-06-17 18:09:16.840 INFO in 'deeppavlov.download'['download'] at line 116: Skipped http://files.deeppavlov.ai/datasets/paraphraser.zip download because of matching hashes
I0617 18:09:16.840891 140398082541376 d

In [5]:
g = para_model.pipe[-1][-1].graph
bert_preprocessor = para_model[0]
bert_classifier = para_model[1]

In [6]:
features_array = bert_preprocessor(texts_a, texts_b)
input_ids = np.array([f.input_ids for f in features_array])
labels = np.array(labels)

In [7]:
from bert_dp import bert

In [8]:
keras_graph = tf.compat.v1.Graph()

In [9]:
def lambda_layer(x):
    with tf.compat.v1.variable_scope('', reuse=tf.AUTO_REUSE):
        output_weights_ = tf.get_variable("output_weights",
                                          [2, 768],
                                          dtype=tf.float32,
                                          initializer=tf.truncated_normal_initializer(stddev=0.02))
        output_bias_ = tf.get_variable("output_bias",
                                       [2],
                                       dtype=tf.float32,
                                       initializer=tf.zeros_initializer())
    logits_ = tf.matmul(x, output_weights_, transpose_b=True)
    logits_ = tf.nn.bias_add(logits_, output_bias_)

#     one_hot_labels_ = tf.one_hot(y_ph, depth=2, dtype=tf.float32)

#     y_predictions_ = tf.argmax(logits_, axis=-1)
#     log_probs_ = tf.nn.log_softmax(logits_, axis=-1)
    y_probas_ = tf.nn.softmax(logits_, axis=-1)
#     per_example_loss_ = -tf.reduce_sum(one_hot_labels_ * log_probs_, axis=-1)
#     loss_ = tf.reduce_mean(per_example_loss_)
    return y_probas_

In [10]:
with keras_graph.as_default():
    keras_sess = tf.compat.v1.Session()
    bm_ = tf.keras.Sequential([bert.BERT(), tf.keras.layers.Lambda(lambda_layer)], name='bert')
    bm_.build(input_shape=(BATCH_SIZE, None))

In [11]:
keras_graph.get_collection('trainable_variables')

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(119547, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32>,
 <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert/encoder/layer_0/attention/self/value/bias:0' shape=(768,) dtype=float32>,
 <tf.

In [12]:
with keras_graph.as_default():
    from tensorflow.python.ops import variables
    keras_vs = variables._all_saveable_objects()
    keras_saver = tf.compat.v1.train.Saver(keras_vs)
    #     keras_sess.run(tf.global_variables_initializer())
    keras_saver_checkpoint_path = 'models/paraphraser_rubert/model_rubert'
    keras_saver.restore(keras_sess, keras_saver_checkpoint_path)

In [13]:
# para_model(['9 мая метрополитен Петербурга будет работать круглосуточно'], ['Петербургское метро в ночь на 10 мая будет работать круглосуточно'])

In [13]:
from tensorflow.keras.metrics import Metric


class F1Score(Metric):
    """Calculates F1 micro, macro or weighted based on the user's choice.
    F1 score is the weighted average of precision and
    recall. Output range is [0, 1]. This works for both
    multi-class and multi-label classification.
    Args:
       num_classes : Number of unique classes in the dataset.
       average : Type of averaging to be performed on data.
                 Acceptable values are None, micro, macro and
                 weighted.
                 Default value is None.
    Returns:
       F1 score: float
    Raises:
       ValueError: If the `average` has values other than
       [None, micro, macro. weighted].
    `average` parameter behavior:
    1. If `None` is specified as an input, scores for each
       class are returned.
    2. If `micro` is specified, metrics like true positivies,
       false positives and false negatives are computed
       globally.
    3. If `macro` is specified, metrics like true positivies,
       false positives and false negatives are computed for
       each class and their unweighted mean is returned.
       Imbalance in dataset is not taken into account for
       calculating the score
    4. If `weighted` is specified, metrics are computed for
       each class and returns the mean weighted by the
       number of true instances in each class taking data
       imbalance into account.
    Usage:
    ```python
    actuals = tf.constant([[1, 1, 0],[1, 0, 0]],
              dtype=tf.int32)
    preds = tf.constant([[1, 0, 0],[1, 0, 1]],
            dtype=tf.int32)
    output = tf.keras.metrics.F1Score(num_classes=3,
              average='micro')
    output.update_state(actuals, predictions)
    print('F1 Micro score is: ',
            output.result().numpy()) # 0.6666667
    ```
    """

    def __init__(self,
                 num_classes,
                 average=None,
                 name='f1_score',
                 dtype=tf.float32):
        super(F1Score, self).__init__(name=name)
        self.num_classes = num_classes
        if average not in (None, 'micro', 'macro', 'weighted'):
            raise ValueError("Unknown average type. Acceptable values "
                             "are: [micro, macro, weighted]")
        else:
            self.average = average
            if self.average == 'micro':
                self.axis = None
            else:
                self.axis = 0
        if self.average == 'micro':
            self.true_positives = self.add_weight(
                'true_positives',
                shape=[],
                initializer='zeros',
                dtype=tf.float32)
            self.false_positives = self.add_weight(
                'false_positives',
                shape=[],
                initializer='zeros',
                dtype=tf.float32)
            self.false_negatives = self.add_weight(
                'false_negatives',
                shape=[],
                initializer='zeros',
                dtype=tf.float32)
        else:
            self.true_positives = self.add_weight(
                'true_positives',
                shape=[self.num_classes],
                initializer='zeros',
                dtype=tf.float32)
            self.false_positives = self.add_weight(
                'false_positives',
                shape=[self.num_classes],
                initializer='zeros',
                dtype=tf.float32)
            self.false_negatives = self.add_weight(
                'false_negatives',
                shape=[self.num_classes],
                initializer='zeros',
                dtype=tf.float32)
            self.weights_intermediate = self.add_weight(
                'weights',
                shape=[self.num_classes],
                initializer='zeros',
                dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.int32)

        # true positive
        self.true_positives.assign_add(
            tf.cast(
                tf.math.count_nonzero(y_pred * y_true, axis=self.axis),
                tf.float32))
        # false positive
        self.false_positives.assign_add(
            tf.cast(
                tf.math.count_nonzero(y_pred * (y_true - 1), axis=self.axis),
                tf.float32))
        # false negative
        self.false_negatives.assign_add(
            tf.cast(
                tf.math.count_nonzero((y_pred - 1) * y_true, axis=self.axis),
                tf.float32))
        if self.average == 'weighted':
            # variable to hold intermediate weights
            self.weights_intermediate.assign_add(
                tf.cast(tf.reduce_sum(y_true, axis=self.axis), tf.float32))

    def result(self):
        p_sum = tf.cast(self.true_positives + self.false_positives, tf.float32)
        # calculate precision
        precision = tf.math.divide_no_nan(self.true_positives, p_sum)

        r_sum = tf.cast(self.true_positives + self.false_negatives, tf.float32)
        # calculate recall
        recall = tf.math.divide_no_nan(self.true_positives, r_sum)

        mul_value = 2 * precision * recall
        add_value = precision + recall
        f1_int = tf.math.divide_no_nan(mul_value, add_value)
        # f1 score
        if self.average is not None:
            f1_score = tf.reduce_mean(f1_int)
        else:
            f1_score = f1_int
        # condition for weighted f1 score
        if self.average == 'weighted':
            f1_int_weights = tf.math.divide_no_nan(
                self.weights_intermediate,
                tf.reduce_sum(self.weights_intermediate))
            # weighted f1 score calculation
            f1_score = tf.reduce_sum(f1_int * f1_int_weights)

        return f1_score

    def get_config(self):
        """Returns the serializable config of the metric."""

        config = {
            "num_classes": self.num_classes,
            "average": self.average,
        }
        base_config = super(F1Score, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def reset_states(self):
        # reset state of the variables to zero
        if self.average == 'micro':
            self.true_positives.assign(0)
            self.false_positives.assign(0)
            self.false_negatives.assign(0)
        else:
            self.true_positives.assign(np.zeros(self.num_classes), np.float32)
            self.false_positives.assign(np.zeros(self.num_classes), np.float32)
            self.false_negatives.assign(np.zeros(self.num_classes), np.float32)
            self.weights_intermediate.assign(
                np.zeros(self.num_classes), np.float32)

In [14]:
with keras_graph.as_default():
#     NUM_EPOCHS = 5
    # VAL_EVERY_N_BATCHES = 52  # ~50, but better suited for paraphraser dataset
    # TRAIN_STEPS_PER_EPOCH = NUM_TRAIN_SAMPLES // BATCH_SIZE
    # VALIDATION_STEPS = 512 // BATCH_SIZE  # calculated by hand for now (~546 samples with droppping remainder)
    LEARNING_RATE = 2e-05
    # learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=LEARNING_RATE,
    #                                                               # n. of batches per epoch * ~NUM_EPOCHS for convergence 
    #                                                               decay_steps=TRAIN_STEPS_PER_EPOCH * NUM_EPOCHS,
    #                                                               end_learning_rate=1e-06)
    # from bert_dp.weight_decay_optimizers import AdamW
    bm_.compile(#optimizer=AdamW(weight_decay=0.01,
    #                                             learning_rate=LEARNING_RATE,
    #                                             epsilon=1e-6),  # as in bert_dp
                optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,
                                                   epsilon=1e-6),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
                         F1Score(num_classes=2, average='macro')
                        ])

In [21]:
with keras_graph.as_default():
    p_ = bm_.predict_classes(input_ids)
    bm_.evaluate(x=input_ids, y=labels, batch_size=BATCH_SIZE)

1899/1899 [==============================] - 5s 2ms/sample - loss: 0.6811 - sparse_categorical_accuracy: 0.5951 - f1_score: 0.0000e+00


In [19]:
p = bert_classifier(features_array)

In [23]:
any(p == p_)

True